# LangChain Parser 解析器解析大模型输出

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai.api_key = os.environ['OPENAI_API_KEY']

## 1 加载模型

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
# 默认 temperature 温度参数是 0.7
chat = ChatOpenAI(temperature=0.0)
chat

## 2 创建 Prompt 模板

构造一个提示模板字符串 `template_string`

In [ ]:
template_string = """把由三个反引号包含的文本 \
翻译成一种 {style} 风格。 \
文本: ```{text}```
"""

导入 LangChain 聊天提示模板，以此来实现模板的重复使用

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
# 设置文本风格
customer_style = """正式普通话 \
用一个平静、尊敬的语调
"""

In [ ]:
# 非正式的中文原文
customer_email = """
我很着急，因为已经研二了还没有发出一篇论文，研究方向还没有同伴一起研究。\
更糟的是，我现在做的项目和预定的研究方向关系并不大。\
我现在需要点建议，老铁。
"""

In [ ]:
customer_messages  = prompt_template.format_messages(
    style=customer_style,
    text=customer_email
)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

In [ ]:
print(customer_messages[0])

In [ ]:
customer_response = chat(customer_messages)
print(customer_response.content)

In [ ]:
# 设置新的文本风格
happy_style = """不标准的普通话 \
用一个乐观、积极的语调
"""

In [ ]:
happy_messages  = prompt_template.format_messages(
    style=happy_style,
    text=customer_email
)
happy_response = chat(happy_messages).content
print(happy_response)

## 3 LangChain 解析器

In [ ]:
from langchain.output_parsers.structured import (
    StructuredOutputParser, ResponseSchema)

In [ ]:
customer_review = """\
这款吹叶机非常神奇。 它有四个设置：\
吹蜡烛、微风、风城、龙卷风。 \
两天后它就到了，正好赶上我和妻子的周年纪念日。 \
我想我的妻子会喜欢它到说不出话来。 \
到目前为止，我是唯一一个使用它的人，而且我一直\
每隔一天早上用它来清理草坪上的叶子。 \
它比其他吹叶机稍微贵一点，\
但我认为它的额外功能是值得的。
"""

In [ ]:
review_template = """\
对于以下文本，请从中提取以下信息：

gift：该商品是作为礼物送给别人的吗？
如果是，则回答 True；如果否或未知，则回答 False。

delivery_days：产品到达需要多少天？ 如果没有找到该信息，则输出 -1。

price_value：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(review_template)

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             type="boolean",
                             description="这件物品是作为礼物送给别人的吗？如果是，则回答 True，如果否或未知，则回答 False。")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      type="number",
                                      description="产品需要多少天才能到达？如果没有找到该信息，则输出 -1。")

price_value_schema = ResponseSchema(name="price_value",
                                    type="string",
                                    description="提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表")
# 以列表类型拼接
response_schemas = [gift_schema, delivery_days_schema, price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# `get_format_instructions()` 获取输出解析器的格式指令
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
messages = prompt.format_messages(
    text=customer_review,
    format_instructions=format_instructions)
print(messages[0].content)

In [ ]:
response = chat(messages).content
print(response)

In [ ]:
output_dict = output_parser.parse(response)
print("输出类型:", type(output_dict), "\n输出内容:", output_dict)

In [ ]:
output_dict.get('gift')